# Why is a random sample an unbiased estimator? 

**The** crucial feature that makes an experiment an experiment is the *intervention* that we perform. But, since we're not only looking to conduct an experiment, but also reliably demonstrate with data that some outcome was *caused* by some action, we're going to need a way to demonstrate this. 

## What does it mean for an estimator to be unbiased? 

In order to understand whether an estimator is biased or unbiased, we've got to be able to make a statment about the *truth*. The most interesting questions that we ask as data scientists typically concern statemetns about population characteristics that we can't *direclty* observe. 

- What feeling did this person have when they wrote this sentence? 
- What *real world object* is in this picture? 
- What is *this persons's* likelihood to take action? 
- What is the *causal* effect of X on Y? 

For each of these circumstances we've got to put together an estimate of the population value. 

Throughout this course, and in most statistics courses, it is convention to denote *estimators* of a parameter value with a "hat". And so, if the true parameter that we're interseted in knowing about is the individual causal effect for some person $D$, $\tau_{D}$, then we might denote the estimate for that person as $\hat{\tau}_{D}$. 

An **unbiased** estimator of an unknown value is an estimator whose *expected value* is equal to the *true* parameter value. And so, in the case of an indiviudal causal effect we might say that an an estimator of an individual causal effect, $\hat{\tau}_{i}$ is unbiased if

\begin{equation} 
E[\hat{\tau}_{i}] = \tau
\end{equation} 

Or equivalently, if

\begin{equation}
E[\hat{\tau}_{i} - \tau] = 0 
\end{equation}

## Is the mean an unbiased estimator of a population parameter? 
Consider the following example: Your section instructor, wanting to choose his or her examples to fit in with the pop-culture references that *you* are most familiar with, wants to asses how old you are. But, the section instructor also is not so impolite as to ask you directly. How could they construct an estimate for your age? 

In [1]:
library(data.table)

class <- data.table( 
  'id'    = 1:15, 
  'names' = LETTERS[1:15], 
  'age'  = sample(24:44, size = 15, replace = TRUE)
  ) 
class[ , you := c(1, rep(0,14))] # indicator for "you";  a single 1 and 14 zeros 
class[ , you := sample(you)] # shuffle that indicator
head(class)

id,names,age,you
1,A,41,0
2,B,33,0
3,C,32,0
4,D,25,0
5,E,32,0
6,F,32,0


So, how old are *you*? 

In [2]:
your_age <- class[you == 1, age]
your_age

[1] 29

How old are the other people in the class? Write the slice that will pull this. 

In [3]:
# ANSWER 
class[you == 0, age]

[1] 41 33 32 25 32 32 28 28 37 25 38 44 29 30

Here is *one* way that I could make a guess about how old you are -- pick one person at random from the class, and them their age, and make that my guess for your age. 

In [4]:
first_estimator <- class[you == 0 , sample(age, 1)]

How far off was I? 

In [5]:
first_error = your_age - first_estimator 
first_error

[1] -3

So, I wasn't **right** on. But, that's just a result of only having done this once. The expectation operator places a probability distribution across each of the possible realizations, and then multiplies by the value of that realization. Then we sum those results. 

If I am sampling one person at random, then that would look like this, the probabiltiy of sampling a single person is:

In [6]:
class[ , prob_sample := (1 / sum(you==0))]
head(class)

id,names,age,you,prob_sample
1,A,41,0,0.07142857
2,B,33,0,0.07142857
3,C,32,0,0.07142857
4,D,25,0,0.07142857
5,E,32,0,0.07142857
6,F,32,0,0.07142857


Each persons contribution to the expectation operator is: 

In [7]:
class[ , p_times_age := prob_sample * age]

And so the expectation of this **estimator**, which comes from sampling one person from the classroom is: 

In [8]:
expectation <- class[ , sum(p_times_age)]
expectation

[1] 34.5

How far off was I? 

In [9]:
your_age - expectation

[1] -5.5

Think about doing this a large number of times, where the 'you' indicator is randomly sampled from the population, and then the estimator of one draw is conducted. To do this, I'm goign to pull all the pieces from the last set into a function, so we can run this function a number of times. 

In [10]:
NSIMS <- 1000

c <- data.table( 
  'id'    = 1:15, 
  'names' = LETTERS[1:15], 
  'age'  = sample(24:44, size = 15, replace = TRUE)
  ) 

class_example <- function(class=c) { 
    class[ , you := c(1, rep(0,14))] # indicator for "you";  a single 1 and 14 zeros 
    class[ , you := sample(you)] # shuffle that indicator
    
    class_mean <- class[you==0, mean(age)]
    your_age   <- class[you==1, age]
    
    diff <- your_age - class_mean
    
    return(diff)
}

diff_vector <- rep(NA, NSIMS)

for(i in 1:NSIMS){ 
  diff_vector[i] <- class_example()
}

In [11]:
sprintf('The difference between your age and the estimator, across these runs is %.4f. That is mighty good!', mean(diff_vector))

[1] "The difference between your age and the estimator, across these runs is -0.0782. That is mighty good!"

# Relate this directly to potential outcomes
How does this relate directly to potential outcoems? Consider the *science* table that is provided to us in table 2.1. 

In [12]:
d <- data.table(
  'id'  = 1:7, 
  'y_0' = c(10,15,20,20,10,15,15), 
  'y_1' = c(15,15,30,15,20,15,30), 
  'tau' = c(5, 0,10,-5,10,0,15)
  )

We can't *actully* see both the potential outcome to control and treatment for any **single** person. But, what if we can come up with an unbiased estimate for the potential outcoems to control for the seven villages? 

Randomly sample 5 of the villages and take the average of their potential outcomes. How close does this come to the true mean of 15? 

In [13]:
control_mean <- '[replace with your code]'

Randomly sample 2 village and take the average of their potential outcomes. How close does this come to the true mean of 20? 

In [14]:
treat_mean <- '[replace with your code]'

Calculate the difference of these. How close does this come to the true treatment effect of 5? 

Note that beacuse we've got pretty small sample sizes here -- 5 in control and 2 in treatment -- we might actually not get that close in any *particular* run of this experiment. But, across all the experiments that we *could* have conducted, we will be dead on.

As we increase the sample size of our experiments, the amount that these estimators in *any one* experiment that we run will produce estimates that are much closer to the true parameter values. We characterize this through standard errors, which you have covered in previous stats classes in the program, but which we'll talk about in detail in week 4. 